In [47]:

import pandas as pd
import sys
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [48]:
sys.path.insert(0,"..")
bremen_trips = pd.read_csv("../data/processed/trips.csv")
bremen_trips = bremen_trips.sort_values(by=['start_time'], ascending=True)
bremen_trips["start_time"] = pd.to_datetime(bremen_trips["start_time"])
bremen_trips = bremen_trips[(bremen_trips['start_lng'] != bremen_trips['end_lng']) | (bremen_trips['start_lat'] != bremen_trips['end_lat'])]

In [49]:

for col in ['start_time']:
    bremen_trips['month'] = pd.DatetimeIndex(bremen_trips['start_time']).month
    bremen_trips['booking_date'] = bremen_trips.start_time.dt.date
    bremen_trips['weekdays'] = pd.DatetimeIndex(bremen_trips['start_time']).weekday

In [50]:
bremen_trips.head()


,bike,bike_type,identification,start_time,end_time,duration_sec,start_lng,start_lat,end_lng,end_lat,start_place,end_place,month,booking_date,weekdays
81821,20685,71,12099440,2019-01-20 00:13:00,2019-01-20 00:23:00,600.0,8.812554,53.076481,8.804828,53.084196,0.0,0.0,1,2019-01-20,6
204560,20935,71,7873316,2019-01-20 00:35:00,2019-01-25 08:00:00,458700.0,8.816798,53.072448,8.811472,53.083167,0.0,2946.0,1,2019-01-20,6
49030,20617,71,12099749,2019-01-20 00:40:00,2019-01-20 00:44:00,240.0,8.801623,53.067878,8.793217,53.064244,0.0,0.0,1,2019-01-20,6
131464,20795,71,12099872,2019-01-20 00:47:00,2019-01-20 00:54:00,420.0,8.814744,53.084841,8.801135,53.094596,0.0,0.0,1,2019-01-20,6
200261,20927,71,12100199,2019-01-20 01:07:00,2019-01-20 01:15:00,480.0,8.815254,53.084302,8.806666,53.094964,0.0,0.0,1,2019-01-20,6


In [51]:
# drop entries with booking durations longer than 2 days
bremen_trips = bremen_trips[bremen_trips['duration_sec'] < 5000]

In [52]:
bremen_trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131731 entries, 81821 to 113816
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   bike            131731 non-null  int64         
 1   bike_type       131731 non-null  int64         
 2   identification  131731 non-null  int64         
 3   start_time      131731 non-null  datetime64[ns]
 4   end_time        131731 non-null  object        
 5   duration_sec    131731 non-null  float64       
 6   start_lng       131731 non-null  float64       
 7   start_lat       131731 non-null  float64       
 8   end_lng         131731 non-null  float64       
 9   end_lat         131731 non-null  float64       
 10  start_place     131731 non-null  float64       
 11  end_place       131731 non-null  float64       
 12  month           131731 non-null  int64         
 13  booking_date    131731 non-null  object        
 14  weekdays        131731 non-null 

In [53]:
data_raw = bremen_trips[['bike','bike_type','identification', 'start_time', 'duration_sec', 'start_lng', 'start_lat', 'start_place', 'month', 'weekdays']]

In [54]:
data_raw['hour'] = data_raw['start_time'].dt.hour
data_raw['min'] = data_raw['start_time'].dt.minute
data_raw['day_of_month'] = data_raw['start_time'].dt.day
data_raw.drop(axis = 1, columns = 'start_time', inplace=True)

C:\Users\TomCLG\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\TomCLG\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\TomCLG\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [55]:
X = data_raw.loc[:, data_raw.columns != 'duration_sec']
y = data_raw['duration_sec']

In [56]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131731 entries, 81821 to 113816
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   bike            131731 non-null  int64  
 1   bike_type       131731 non-null  int64  
 2   identification  131731 non-null  int64  
 3   start_lng       131731 non-null  float64
 4   start_lat       131731 non-null  float64
 5   start_place     131731 non-null  float64
 6   month           131731 non-null  int64  
 7   weekdays        131731 non-null  int64  
 8   hour            131731 non-null  int64  
 9   min             131731 non-null  int64  
 10  day_of_month    131731 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 12.1 MB


In [57]:
from sklearn.model_selection import train_test_split


X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.60, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.60, shuffle=True)

In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [95]:
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)

In [96]:
lin = LinearRegression()

In [97]:
lin.fit(X_train_poly, y_train)
lin.score(X_train_poly, y_train)

0.03836359770722719